#Customers Data

## Process the Customers Data
- Ingest the data into the data lakehouse - bronze customers
- Perform the data quality checks and transform the data as required - silver_customers_clean
- Apply the changes to the customers data - silver_customers

###1. Ingest the data into the data lakehouse - Bronze Customers

In [0]:
USE CATALOG EcommerceDataset;

In [0]:
CREATE OR REFRESH STREAMING TABLE ecommercedataset.bronzelayer.CustomersBronzeTable
COMMENT 'Raw Customers data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS 
  SELECT *,
         _metadata.file_path AS input_file_path,
         CURRENT_TIMESTAMP() AS ingestion_timestamp
  FROM STREAM cloud_files(
    '/Volumes/ecommercedataset/landinglayer/landingdata/',
    'json'
  );


message
The operation was successfully executed.


###2. Specifying the Expectations for the customers table 
> Silver Customer Clean (DQ Rules)
1. Fail if customer_id is NULL
2. Drop records with customer_name as NULL
3. Warn if telephone is less than 10 characters 
4. Warn if email is NULL 
5. Warn if date_of_birth is before 1920

> Silver Customer Clean (Transformations)
1. CAST date_of_birth to DATE
2. CAST created_date to DATE

In [0]:
CREATE OR REFRESH STREAMING TABLE ecommercedataset.silverlayer.CustomerSilverClean
-- Setting up the expectations
(
  CONSTRAINT valid_customer_id EXPECT (CUSTOMER_ID IS NOT NULL) ON VIOLATION FAIL UPDATE, 
  CONSTRAINT valid_customer_name EXPECT (CUSTOMER_NAME IS NOT NULL) ON VIOLATION DROP ROW, 
  CONSTRAINT valid_telephone_number EXPECT (LENGTH(telephone) >= 10), 
  CONSTRAINT valid_email EXPECT (email IS NOT NULL), 
  CONSTRAINT valid_date_of_birth EXPECT (date_of_birth >= '1920-01-01')
)
COMMENT 'Table created wiht Data Quality Expectations'
TBLPROPERTIES ('quality' = 'silver')
AS 
SELECT 
    customer_id, 
    customer_name, 
    CAST(date_of_birth AS DATE) as date_of_birth, 
    telephone, 
    email, 
    CAST(created_date AS DATE) as created_date
FROM STREAM(LIVE.CustomersBronzeTable) -- The following STREAM() is used to only the data that is changed in the stream 

###3. Apply changes to the Customers data - Silver Customer

In [0]:
CREATE OR REFRESH STREAMING TABLE ecommercedataset.silverlayer.CustomerSilverSCD
COMMENT 'SCD Type 1 Customers Data'
TBLPROPERTIES ('quality' = 'silver');

In [0]:
APPLY CHANGES INTO LIVE(ecommercedataset.silverlayer.CustomerSilverSCD)
FROM STREAM(LIVE.CustomerSilverClean)
KEYS (customer_id)
SEQUENCE BY created_date
STORED AS SCD TYPE 1; 